# Data Science - Laboratorio 9
## Visualizaciones interactivas y dashboards
### Modelos LSTM
---
**Integrantes**
- Diego Alberto Leiva
- José Pablo Orellana

## Librerias

In [1]:
# Manipulacion de Datos
import pandas as pd
import numpy as np

# Sistema
import os

# Utilidades
import random

# Preprocesamiento
from sklearn.preprocessing import StandardScaler

# PyTorch CUDA
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

### Carga de Datos Limpios

In [2]:
# Leer los archivos de una carpeta y almacenarlos como dataframes
consumos = pd.read_csv('data/consumos_clean.csv', sep=';', encoding='utf-8')
importaciones = pd.read_csv('data/importaciones_clean.csv', sep=';', encoding='utf-8')
precios = pd.read_csv('data/precios_clean.csv', sep=';', encoding='utf-8')

## Modelos LSTM

### Configuracion de PyTorch con CUDA

Se verifica si el sistema tiene capacidad de CUDA o no

In [3]:
# Configurar la semilla para la reproducibilidad
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available():
    print(f"CUDA AVAILABLE")

    print(f"Using: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    # Configurar el generador de números aleatorios de CUDA
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Multi-GPU.

    # Configurar PyTorch a determinista
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
else:
    # Si no hay GPU disponible, usar la CPU
    print("CUDA NOT AVAILABLE")
    print("Using CPU")

CUDA AVAILABLE
Using: NVIDIA GeForce RTX 3070 Ti Laptop GPU


### Arquitectura de una LSTM con PyTorch CUDA

In [4]:
class LSTM_Model():
    """
    Clase para entrenar un modelo LSTM para predecir series temporales.

    Args:
        hidden_layers (int): Número de capas ocultas.
        num_layers (int): Número de capas LSTM.
        lookback (int): Número de pasos de tiempo hacia atrás.
        n_epochs (int): Número de épocas.
        test_size (int): Tamaño del conjunto de prueba.
        dataframe (pandas.DataFrame): Dataframe de entrada.
        target_column (str): Columna objetivo.

    Attributes:
        hidden_layers (int): Número de capas ocultas.
        num_layers (int): Número de capas LSTM.
        lookback (int): Número de pasos de tiempo hacia atrás.
        n_epochs (int): Número de épocas.
        test_size (int): Tamaño del conjunto de prueba.
        dataframe (pandas.DataFrame): Dataframe de entrada.
        target_column (str): Columna objetivo.
        model (torch.nn.Module): Modelo LSTM.
        loss_fn (torch.nn.Module): Función de pérdida.
        optimizer (torch.optim.Optimizer): Optimizador.
        train_size (int): Tamaño del conjunto de entrenamiento.
    """
    def __init__(self, hidden_layers, num_layers, lookback, n_epochs, test_size, dataframe, target_column):
        
        # Parámetros fijos
        self.input_size = 1                    # Fijo, tamaño de entrada
        self.output_size = 1                   # Fijo, tamaño de salida
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Dispositivo fijo
        
        # Inicializar otros hiperparámetros
        self.hidden_layers = hidden_layers     # Número de capas ocultas
        self.num_layers = num_layers           # Número de capas LSTM
        self.lookback = lookback               # Número de pasos de tiempo hacia atrás
        self.n_epochs = n_epochs               # Número de épocas
        self.test_size = test_size             # Tamaño del conjunto de prueba
        self.dataframe = dataframe             # Dataframe de entrada
        self.target_column = target_column     # Columna objetivo

        # Crear el modelo LSTM 
        self.model = self.build_lstm_model()

        # Definir la función de pérdida y el optimizador
        self.loss_fn = nn.MSELoss().to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

        # Entrenar el modelo
        self.train_preds, self.test_preds = self.train_model(dataframe, target_column)

    def build_lstm_model(self):
        """
        Función para construir el modelo LSTM.

        Returns:
            model (torch.nn.Module): El modelo LSTM.
        """
        class LSTM(nn.Module):
            def __init__(self, input_size, hidden_layers, output_size, num_layers):
                super(LSTM, self).__init__()

                # Definir la capa LSTM
                self.lstm = nn.LSTM(input_size=input_size, 
                                    hidden_size=hidden_layers, 
                                    num_layers=num_layers, 
                                    batch_first=True)

                # Definir la capa de salida
                self.linear = nn.Linear(hidden_layers, output_size)

                # Capa de activación ReLU
                self.relu = nn.ReLU()
                
            def forward(self, x):
                # Inicializar hidden states
                h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
                c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)

                # Forward pass a través de LSTM
                out, _ = self.lstm(x, (h0, c0))

                # Aplicar la capa de activación
                out = self.relu(out)

                # Solo el último hidden state
                out = self.linear(out[:, -1, :])
                
                # Devolver la predicción
                return out

        # Crear el modelo LSTM con los hiperparámetros definidos
        return LSTM(self.input_size, self.hidden_layers, self.output_size, self.num_layers).to(self.device)


    def prepare_data(self, dataset):
        """
        Función para crear el dataset de entrenamiento.

        Args:
            dataset (numpy.array): El dataset de entrada.

        Returns:
            X (torch.Tensor): El tensor de features.
            y (torch.Tensor): El tensor de target.
        """
        X, y = [], []
        for i in range(len(dataset) - self.lookback):
            X.append(dataset[i:i + self.lookback])
            y.append(dataset[i + self.lookback])

        # Convertir listas a numpy arrays y luego a tensores
        X = np.array(X)
        y = np.array(y)

        # Retornar tensores
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


    def train_model(self, dataframe, variable):
        """
        Función para entrenar el modelo LSTM.

        Args:
            dataframe (pandas.DataFrame): El dataframe de entrada.

        Returns:
            pred_train (numpy.array): Predicciones sobre el conjunto de entrenamiento.
            pred_test (numpy.array): Predicciones sobre el conjunto de prueba.
        """
        # Preparar los datos
        df = dataframe.copy()
        timeseries = df[[variable]].values.astype(float)

        # Normalizar los datos
        scaler = StandardScaler()
        timeseries = scaler.fit_transform(timeseries)

        # Dividir en entrenamiento y prueba
        self.train_size = len(timeseries) - self.test_size
        train_data = timeseries[:self.train_size]
        test_data = timeseries[self.train_size:]

        # Crear datasets
        X_train, y_train = self.prepare_data(train_data)
        X_test, y_test = self.prepare_data(test_data)

        # Mover los tensores al dispositivo
        X_train, y_train = X_train.to(self.device), y_train.to(self.device)
        X_test, y_test = X_test.to(self.device), y_test.to(self.device)

        # Crear DataLoader para entrenamiento
        loader = data.DataLoader(data.TensorDataset(X_train, y_train), batch_size=16, shuffle=False)

        # Entrenar el modelo
        for epoch in range(self.n_epochs):
            self.model.train()
            for X_batch, y_batch in loader:
                X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)

                # Predicción
                y_pred = self.model(X_batch)

                # Calcular pérdida
                loss = self.loss_fn(y_pred, y_batch)

                # Backpropagation y optimización
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

        # Predicciones sobre el conjunto de entrenamiento y prueba
        self.model.eval()
        with torch.no_grad():
            pred_train = self.model(X_train).cpu().numpy()
            pred_test = self.model(X_test).cpu().numpy()

        # Retornar predicciones
        return pred_train, pred_test


In [5]:
# Definir los parámetros del modelo
hidden_layers = 50    # Número de capas ocultas en el LSTM
num_layers = 2        # Número de capas LSTM
lookback = 10         # Número de pasos de tiempo hacia atrás
n_epochs = 100        # Número de épocas de entrenamiento
test_size = 20        # Tamaño del conjunto de prueba
target_column = 'Diesel'  # Nombre de la columna objetivo en el dataframe

# Crear la instancia del modelo
lstm_model = LSTM_Model(
    hidden_layers=hidden_layers,
    num_layers=num_layers,
    lookback=lookback,
    n_epochs=n_epochs,
    test_size=test_size,
    dataframe=consumos,
    target_column=target_column
)

In [6]:
prediction_train = lstm_model.train_preds
print(prediction_train)

[[-8.9068294e-01]
 [-7.4541962e-01]
 [-6.4456367e-01]
 [-8.4119332e-01]
 [-1.0311841e+00]
 [-9.5345080e-01]
 [-9.5798314e-01]
 [-8.6620861e-01]
 [-9.9201602e-01]
 [-1.0771925e+00]
 [-1.1276046e+00]
 [-1.1707462e+00]
 [-1.1132865e+00]
 [-8.7282825e-01]
 [-6.9056773e-01]
 [-5.5293566e-01]
 [-4.3525884e-01]
 [-6.1242479e-01]
 [-8.1535387e-01]
 [-9.0574479e-01]
 [-9.4920361e-01]
 [-9.5472473e-01]
 [-9.2982197e-01]
 [-8.6957490e-01]
 [-8.2641017e-01]
 [-8.2219869e-01]
 [-7.5042689e-01]
 [-5.2509481e-01]
 [-4.3416855e-01]
 [-3.5491329e-01]
 [-4.9006057e-01]
 [-6.3801956e-01]
 [-8.9397436e-01]
 [-1.0048161e+00]
 [-1.0706835e+00]
 [-1.0839605e+00]
 [-6.6359967e-01]
 [-5.7552767e-01]
 [-5.3952974e-01]
 [-6.4196658e-01]
 [-7.3402262e-01]
 [-7.4702835e-01]
 [-8.1535435e-01]
 [-1.0308228e+00]
 [-1.1313249e+00]
 [-1.1451058e+00]
 [-9.8540640e-01]
 [-7.8131759e-01]
 [-7.1521962e-01]
 [-6.2061965e-01]
 [-4.6598127e-01]
 [-4.7671983e-01]
 [-5.5162168e-01]
 [-4.8981684e-01]
 [-4.4473416e-01]
 [-6.14232

In [7]:
prediction_test = lstm_model.test_preds
print(prediction_test)

[[1.0954587]
 [1.2674663]
 [1.347435 ]
 [1.1610581]
 [1.4934587]
 [1.9609854]
 [2.0408938]
 [2.042473 ]
 [2.236739 ]
 [2.0888798]]
